In [1]:
import os
from dotenv import load_dotenv, find_dotenv
from openai import OpenAI
import instructor
from pydantic import BaseModel, Field
from typing import Optional, List
from rich import print as rprint

load_dotenv(find_dotenv())

True

In [2]:
client_image = instructor.patch(OpenAI(api_key=os.getenv("OPENAI_API_KEY")), mode=instructor.Mode.MD_JSON)
client_copy = instructor.patch(OpenAI(api_key=os.getenv("OPENAI_API_KEY")), mode=instructor.Mode.FUNCTIONS)

In [3]:
class Product(BaseModel):
    """
    Represents a product extracted from an image using AI.

    The product attributes are dynamically determined based on the content
    of the image and the AI's interpretation. This class serves as a structured
    representation of the identified product characteristics.
    """

    name: str = Field(
        description="A generic name for the product.", example="Headphones"
    )
    key_features: Optional[list[str]] = Field(
        description="A list of key features of the product that stand out.",
        example=["Wireless", "Noise Cancellation"],
        default=None,
    )

    description: Optional[str] = Field(
        description="A description of the product.",
        example="Wireless headphones with noise cancellation.",
        default=None,
    )

    def generate_prompt(self):
        prompt = f"Product: {self.name}\n"
        if self.description:
            prompt += f"Description: {self.description}\n"
        if self.key_features:
            prompt += f"Key Features: {', '.join(self.key_features)}\n"
        return prompt
    
    def __repr__(self):
        return self.generate_prompt()

class IdentifiedProduct(BaseModel):
    """
    Represents a list of products identified in the image.
    """

    products: Optional[List[Product]] = Field(
        description="A list of products identified by the AI.",
        example=[
            Product(
                name="Headphones",
                description="Wireless headphones with noise cancellation.",
                key_features=["Wireless", "Noise Cancellation"],
            )
        ],
        default=None,
    )

    error: bool = Field(default=False)
    message: Optional[str] = Field(default=None)

    def __bool__(self):
        return self.products is not None and len(self.products) > 0


In [4]:
def read_images(image_urls: List[str]):
    return client_image.chat.completions.create(
        model="gpt-4-vision-preview",
        response_model=IdentifiedProduct,
        max_tokens=1024,
        temperature=0,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Identify a product using the given images and generate key features",
                    },
                    *[
                        {"type": "image_url", "image_url": {"url": url}}
                        for url in image_urls
                    ],
                ],
            }
        ],
    )


In [5]:
class AdCopy(BaseModel):
    """
    Represents a generated ad copy.
    """

    ad_copy: str = Field(
        description="A long-form advertisement copy for a product. This will be used in campaigns to promote the product with a persuasive message and a call-to-action with the objective of driving sales.",
        example="""
        "Experience the ultimate sound quality with our wireless headphones, featuring high-definition audio, noise-cancellation, and a comfortable, ergonomic design for all-day listening."
        """,
    )
    name: str = Field(
        description="The name of the product being advertised.",
        example="Headphones",
    )

    def __str__(self):
        return f"{self.name}: {self.ad_copy}"

    def __repr__(self):
        return str(self)


In [6]:
def generate_ad_copy(product: Product):
    return client_copy.chat.completions.create(
        model="gpt-4-1106-preview",
        response_model=AdCopy,
        temperature=0.1,
        messages=[
            {
                "role": "system",
                "content": "You are an expert marketing assistant for all products. Your task is to generate an advertisement copy for a product using the name, description, and key features.",
            },
            {"role": "user", "content": product.generate_prompt()},
        ],
    )


In [7]:
def run(images: list[str]) -> List[AdCopy]:
    """
    Given a list of images, identify the products in the images and generate ad copy for each product.
    """

    identified_products: IdentifiedProduct = read_images(images)
    ad_copies = []

    if identified_products.error:
        rprint(f"[red]Error: {identified_products.message}[/red]")
        return []

    if not identified_products:
        rprint("[yellow]No products identified.[/yellow]")
        return []

    rprint("[green]Products identified:[/green]")

    for product in identified_products.products:
        rprint(f"[green]{product}[/green]")
        ad_copy: AdCopy = generate_ad_copy(product)
        rprint(f"[blue]Ad Copy: {ad_copy.ad_copy}[/blue]")
        ad_copies.append(ad_copy)

    return {"products": identified_products.products, "ad_copies": ad_copies}


In [8]:
images = [
    "https://bruvi.com/cdn/shop/files/StandAloneBrewer_850x.png?v=1691162569",
    "https://bruvi.com/cdn/shop/files/Pod_Insert_Bundle_e9c2e60a-e112-4a8b-8ee3-674b5ba7958b_850x.png?v=1696286536",
    "https://bruvi.com/cdn/shop/files/Joyride-PodBox_550x.png?v=1682464498"
]


In [11]:
x = run(images)

Products identified:

name='Single-Serve Coffee Maker' key_features=['Pod-based brewing system', 'Multiple drink options', 'Compact 
design'] description='A single-serve coffee maker that uses pods to brew a variety of coffee and tea beverages.'

Ad Copy: Introducing the ultimate convenience in your daily routine – the Single-Serve Coffee Maker! Say goodbye to
the hassle of brewing a whole pot when all you need is one perfect cup. Our pod-based brewing system offers you a 
world of multiple drink options at the touch of a button. Whether you're craving a bold espresso, a classic cup of 
coffee, or a soothing tea, your desires are covered. Its sleek, compact design fits seamlessly into any kitchen 
space, making it the ideal choice for coffee lovers living in cozy city apartments or for those who appreciate 
clutter-free countertops. Elevate your coffee experience with the Single-Serve Coffee Maker – where every sip is a 
fresh adventure. Order now and transform your mornings into moments of pure bliss!

name='Coffee Pods' key_features=['Pre-measured coffee', 'Convenient', 'Variety of flavors'] 
description='Pre-packaged coffee pods for use with compatible single-serve coffee makers, offering a convenient way
to enjoy a variety of coffee flavors.'

Ad Copy: Wake up to the simplicity of flavor with our premium Coffee Pods! Designed for coffee lovers who value 
convenience without compromising on taste, our Coffee Pods are the perfect blend of efficiency and indulgence. Each
pod is pre-measured to ensure the perfect cup every time, so you can savor your favorite coffee flavors with ease. 
Whether you're in the mood for a bold espresso or a smooth vanilla, our variety of flavors caters to every palate. 
Say goodbye to messy coffee grounds and hello to a cleaner, quicker coffee experience. Transform your morning 
routine with our Coffee Pods – just pop, brew, and enjoy! Order now and discover the ultimate coffee convenience.